將annotations資料夾裡的xml轉換成csv檔

In [1]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [15]:
categories_list = ['raccoon', 'kangaroo']
categories_path = []
annotations_path = []
images_path = []

for i in range(len(categories_list)):
    categories_path.append(os.path.join(os.getcwd(), 'Day049-050\\' + categories_list[i] + '\\'))
    annotations_path.append(os.path.join(os.getcwd(), 'Day049-050\\' + categories_list[i] + '\\annotations\\'))
    images_path.append(os.path.join(os.getcwd(), 'Day049-050\\' + categories_list[i] + '\\images\\'))

for i in range(len(categories_list)):
    destination_path = categories_path[i] + categories_list[i] + '_labels.csv'
    if not os.path.exists(destination_path):
        xml_df = xml_to_csv(annotations_path[i])
        xml_df.to_csv(destination_path, index=None)
        print('Successfully converted ' + categories_list[i] + ' xml to csv.')
    else:
        print(categories_list[i] + '_labels.csv already exists')

raccoon_labels.csv already exists
kangaroo_labels.csv already exists


將資料格式轉為yolo3格式

In [24]:
for i in range(len(categories_list)):
    source_path = categories_path[i] + categories_list[i] + '_labels.csv'
    destination_path = categories_path[i] + categories_list[i] + '_yolo3.txt'
    if not os.path.exists(destination_path):
        labels_file = open(source_path, 'r')
        labels_line = labels_file.readline() #讀取標題

        yolo3_file = open(destination_path, 'w')
        
        labels_line = labels_file.readline() #讀取第一筆資料
        data = labels_line[:-1].split(',')
        last_file_name = data[0]
        yolo3_file.write(images_path[i] + last_file_name + ' ' + data[4] + ',' + data[5] + ',' + data[6] + ',' + data[7] + ',' + str(i))

        while labels_line:
            labels_line = labels_file.readline()
            if labels_line == '':
                break
            data = labels_line[:-1].split(',')
            
            if last_file_name != data[0]:
                last_file_name = data[0]
                yolo3_file.write('\n')
                yolo3_file.write(images_path[i] + last_file_name + ' ' + data[4] + ',' + data[5] + ',' + data[6] + ',' + data[7] + ',' + str(i))
            else:
                yolo3_file.write(' ' + data[4] + ',' + data[5] + ',' + data[6] + ',' + data[7] + ',' + str(i))
                
        yolo3_file.close()
        print('Successfully converted ' + categories_list[i] + ' csv to yolo3.')
    else:
        print(categories_list[i] + '_yolo3.txt already exists')

Successfully converted raccoon csv to yolo3.
Successfully converted kangaroo csv to yolo3.
